# Named Entity Recognition

## Set variables

In [1]:
file_name = "COPIOUS_ELECTRA.ipynb"
batch_size = 16
dataset = "COPIOUS"
data_directory = "../Datasets/NER/COPIOUS-txt/"
model_src = "google/electra-base-discriminator"

## Install and import necessary libraries

In [2]:
# !pip install transformers
# !pip install datasets
# !pip install seqeval
# !pip install accelerate -U

In [3]:
import os
import shutil
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Convert the dataset to CoNLL2003 format

In [7]:
def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""

    # read file
    lines =  open(filename).read().strip()

    # find sentence-like boundaries
    lines = lines.split("\n\n")

     # split on newlines
    lines = [line.split("\n") for line in lines]

    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]

    # get labels/tags
    labels = [[l.split()[idx] for l in line] for line in lines]

    #convert to df
    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)

    return df

In [8]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [9]:
DATADIR = data_directory

def get_data(trainfile=DATADIR + "train.txt",
             devfile=DATADIR + "dev.txt",
             testfile=DATADIR + "test.txt"):

    train = read_CoNLL2003_format(trainfile, idx=3)
    dev = read_CoNLL2003_format(devfile, idx=3)
    print("Train data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

    print("Dev data: %d sentences, %d tokens"%(len(dev),len(flatten(dev.tokens))))

    test = read_CoNLL2003_format(testfile, idx=3)
    print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))

    return train, test, dev

In [10]:
train, test, dev = get_data()

train_dataset = Dataset.from_pandas(train)
dev_dataset = Dataset.from_pandas(dev)
test_dataset = Dataset.from_pandas(test)

Train data: 23695 sentences, 313311 tokens
Dev data: 4101 sentences, 37218 tokens
Test data: 3362 sentences, 37339 tokens


## Tokenize the dataset

In [11]:
label_list = ['B-GeographicalLocation', 'B-Habitat', 'B-Person', 'B-Taxon', 'B-TemporalExpression', 'I-GeographicalLocation', 'I-Habitat', 'I-Person', 'I-Taxon', 'I-TemporalExpression', 'O']
label2id = {k: v for v, k in enumerate(label_list)}
id2label = {v: k for v, k in enumerate(label_list)}
label2id

{'B-GeographicalLocation': 0,
 'B-Habitat': 1,
 'B-Person': 2,
 'B-Taxon': 3,
 'B-TemporalExpression': 4,
 'I-GeographicalLocation': 5,
 'I-Habitat': 6,
 'I-Person': 7,
 'I-Taxon': 8,
 'I-TemporalExpression': 9,
 'O': 10}

In [12]:
task = "ner"
model_checkpoint = model_src

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [13]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), max_length= 512, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized_datasets = dev_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/23695 [00:00<?, ? examples/s]

Map: 100%|██████████| 3362/3362 [00:00<00:00, 4446.94 examples/s]


## Finetuning the model to the dataset

In [14]:
model =  AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label, label2id=label2id)

args = TrainingArguments(
    f"test-{task}",
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    num_train_epochs=3,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=dev_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1728517/133569150.py:16: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


Clear model directory if it hasn't been cleared yet.

In [15]:
directory_path = "./ner.model"

if os.path.exists(directory_path):
    try:
        # Remove the directory
        shutil.rmtree(directory_path)
        print(f"Directory '{directory_path}' removed successfully.")
    except Exception as e:
        print(f"Error removing directory '{directory_path}': {e}")
else:
    print(f"Directory '{directory_path}' does not exist.")

Directory './ner.model' removed successfully.


In [16]:
trainer.train()
trainer.evaluate()
trainer.save_model('ner.model')

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.149800,0.242737,0.703704,0.836431,0.764348,0.934467
2,0.154900,0.173255,0.798654,0.859127,0.827788,0.953495
3,0.063600,0.177153,0.808859,0.861084,0.834155,0.956602


/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along

## Making an inference with the finetuned model

In [17]:
from transformers import pipeline

Change model_checkpoint as needed

In [18]:
model_checkpoint = "./ner.model"

In [19]:
token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")

In [20]:
token_classifier("Birgus latro is widely distributed throughout the Western Pacific and eastern Indian Oceans")

[{'entity_group': 'Taxon',
  'score': 0.9956707,
  'word': 'birgus latro',
  'start': 0,
  'end': 12},
 {'entity_group': 'GeographicalLocation',
  'score': 0.98754984,
  'word': 'western pacific',
  'start': 50,
  'end': 65},
 {'entity_group': 'GeographicalLocation',
  'score': 0.98964137,
  'word': 'eastern indian oceans',
  'start': 70,
  'end': 91}]

## Get precision, f1-score, and recall for each entity group

In [21]:
predictions, labels, _ = trainer.predict(test_tokenized_datasets)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'GeographicalLocation': {'precision': 0.8490432317505315,
  'recall': 0.8907063197026023,
  'f1': 0.8693759071117562,
  'number': 1345},
 'Habitat': {'precision': 0.6186046511627907,
  'recall': 0.7074468085106383,
  'f1': 0.6600496277915632,
  'number': 188},
 'Person': {'precision': 0.7186761229314421,
  'recall': 0.6565874730021598,
  'f1': 0.6862302483069977,
  'number': 463},
 'Taxon': {'precision': 0.8528761061946902,
  'recall': 0.8855283307810107,
  'f1': 0.8688955672426746,
  'number': 2612},
 'TemporalExpression': {'precision': 0.7785714285714286,
  'recall': 0.8449612403100775,
  'f1': 0.8104089219330853,
  'number': 258},
 'overall_precision': 0.8264233287046221,
 'overall_recall': 0.8561446773530621,
 'overall_f1': 0.8410214999495307,
 'overall_accuracy': 0.9597474869674241}